# Deep Learning Toolkit for Splunk - SentenceBERT Notebook

This notebook contains an example workflow how to use SentenceBERT model to calculate the cosine similarity between two sentences with the Deep Learning Toolkit for Splunk.

Note: By default every time you save this notebook the cells are exported into a python module which is then invoked by Splunk MLTK commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the Deep Learning Toolkit app for more information.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [24]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
from transformers import pipeline
from transformers import AutoTokenizer, AutoModel
from transformers import BertJapaneseTokenizer, BertTokenizer, BertModel
import torch
# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)

numpy version: 1.22.1
pandas version: 2.0.3


## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. Note: mode=stage is used in the | fit command to do this.

| makeresults
| eval text1="これは本です", text2="初めまして"
| fit MLTKContainer mode=stage algo=transformers_sentencebert lang=jp from text1 text2 into app:transformers_sentencebert

For the input parameters, "lang" specifies the language of the inputs, where Japanese ("jp") and English ("en", defaulted) are supported. Input fields containing the two sentences to be compared should be named "text1" and "text2".

In [5]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [21]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("transformers_sentencebert")
print(df.describe())
print(param)

              text1    text2
count             1        1
unique            1        1
top     Hello there  I am AI
freq              1        1
{'options': {'params': {'mode': 'stage', 'algo': 'transformers_sentencebert', 'lang': 'en'}, 'feature_variables': ['text1', 'text2'], 'args': ['text1', 'text2'], 'model_name': 'transformers_sentencebert', 'algo_name': 'MLTKContainer', 'mlspl_limits': {'handle_new_cat': 'default', 'max_distinct_cat_values': '100', 'max_distinct_cat_values_for_classifiers': '100', 'max_distinct_cat_values_for_scoring': '100', 'max_fit_time': '600', 'max_inputs': '100000', 'max_memory_usage_mb': '4000', 'max_model_size_mb': '30', 'max_score_time': '600', 'use_sampling': 'true'}, 'kfold_cv': None}, 'feature_variables': ['text1', 'text2']}


## Stage 2 - create and initialize a model

In [ ]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = {}
    model['hyperparameter'] = 42.0
    return model

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(init(df,param))

## Stage 3 - fit the model

In [ ]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    # model.fit()
    info = {"message": "model trained"}
    return info

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

## Stage 4 - apply the model

Examples: 
| makeresults
| eval text1="これは本です", text2="初めまして"
| fit MLTKContainer algo=transformers_sentencebert lang=jp from text1 text2 into app:transformers_sentencebert

In [25]:
class SentenceBertJapanese:
    def __init__(self, model_name_or_path, device=None):
        self.tokenizer = BertJapaneseTokenizer.from_pretrained(model_name_or_path)
        self.model = BertModel.from_pretrained(model_name_or_path)
        self.model.eval()

        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = torch.device(device)
        self.model.to(device)

    def _mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


    def encode(self, sentences, batch_size=8):
        all_embeddings = []
        iterator = range(0, len(sentences), batch_size)
        for batch_idx in iterator:
            batch = sentences[batch_idx:batch_idx + batch_size]

            encoded_input = self.tokenizer.batch_encode_plus(batch, padding="longest", 
                                           truncation=True, return_tensors="pt").to(self.device)
            model_output = self.model(**encoded_input)
            sentence_embeddings = self._mean_pooling(model_output, encoded_input["attention_mask"]).to('cpu')

            all_embeddings.extend(sentence_embeddings)

        # return torch.stack(all_embeddings).numpy()
        return torch.stack(all_embeddings)

class SentenceBert:
    def __init__(self, model_name_or_path, device=None):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
        self.model = AutoModel.from_pretrained(model_name_or_path)
        self.model.eval()

        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = torch.device(device)
        self.model.to(device)

    def _mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


    def encode(self, sentences, batch_size=8):
        all_embeddings = []
        iterator = range(0, len(sentences), batch_size)
        for batch_idx in iterator:
            batch = sentences[batch_idx:batch_idx + batch_size]

            encoded_input = self.tokenizer.batch_encode_plus(batch, padding="longest", 
                                           truncation=True, return_tensors="pt").to(self.device)
            model_output = self.model(**encoded_input)
            sentence_embeddings = self._mean_pooling(model_output, encoded_input["attention_mask"]).to('cpu')

            all_embeddings.extend(sentence_embeddings)

        # return torch.stack(all_embeddings).numpy()
        return torch.stack(all_embeddings)
        

# apply your model
# returns the calculated results
def apply(model,df,param):
    try:
        if param['options']['params']['lang'] == "jp":
            MODEL_NAME = "sonoisa/sentence-bert-base-ja-mean-tokens-v2"
            model = SentenceBertJapanese(MODEL_NAME)
        else:
            MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
            model = SentenceBert(MODEL_NAME)
    except:
        MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
        model = SentenceBert(MODEL_NAME)

    X1 = df.text1.values
    X2 = df.text2.values
    Y = []
    for i in range(len(X1)):
        pair = [X1[i], X2[i]]
        sentence_vectors = model.encode(pair)
        A, B = sentence_vectors[0].detach(), sentence_vectors[-1].detach()
        cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
        output = cos(A, B)
        Y.append(round(float(output),2))

    cols={"similarity score": Y}
    result=pd.DataFrame(data=cols)
    
    return result

In [26]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(None,df,param))

   similarity score
0               0.3


## Stage 5 - save the model

In [ ]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    with open(MODEL_DIRECTORY + name + ".json", 'w') as file:
        json.dump(model, file)
    return model

## Stage 6 - load the model

In [ ]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = {}
    with open(MODEL_DIRECTORY + name + ".json", 'r') as file:
        model = json.load(file)
    return model

## Stage 7 - provide a summary of the model

In [ ]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code